# data processing notebook

This notebook performs basic geospatial operations to convert between data types and prepare all the materials for modeling to begin.

Steps:

* Compute average tree heights on a per-pixel basis
    * sum all tree height measurements from each location
    * divide by the total number of measurements made
* Stack and consistently mask the covariate datasets
* Apply that same mask to the tree height data
* Split the covariate stack into smaller tiles to manage memory when applying the full model

In [ ]:
# import packages
import os
import ccb
import gdal
import numpy as np
import otbApplication as otb

# raise gdal runtime errors
gdal.UseExceptions()

# set the working directory
wd = 'data'

# set the nodata value
nodata = -9999

# set the output tile size
tile_size = 1024

In [ ]:
# set the vector we'll compute height values from
vector_path = os.path.join(wd, 'lvis-lidar-pts-32616.shp')

# set the raster that determines the spatial extent and pixel size of the output files
raster_path = os.path.join(wd, 'tree-cover.tif')

# read the raster metadata
raster = ccb.read.raster(raster_path)

## Computing average tree heights
Everything here assumes the input raster and vector are the same projection. Double check that this is still true if you're changing any of the input files!

In [ ]:
# first, create a temporary data directory to store outputs
td = os.path.join(wd, 'temp')
if not os.path.exists(td):
    os.mkdir(td)

In [ ]:
# next, compute the sum of all height measurements
height_sum_path = os.path.join(td, 'lvis-height-sum.tif')
rasterize_options = gdal.RasterizeOptions(
    format = 'GTiff',
    outputType = gdal.GDT_Float32,
    creationOptions = ['COMPRESS=DEFLATE', 'TILED=YES'],
    noData = nodata,
    initValues = 0,
    xRes = raster.xps,
    yRes = np.abs(raster.yps),
    outputBounds = [
        raster.xmin,
        raster.ymin,
        raster.xmax,
        raster.ymax
    ],
    attribute = 'tree-heigh',
    options = ['-add']
)

# then run the rasterization command
print('computing sum of tree height values')
print('estimated time: 30s')
ref = gdal.Rasterize(
    height_sum_path,
    vector_path,
    options = rasterize_options
)

# and write to disk
ref.FlushCache()
print('done!')

In [ ]:
# next, compute the count of all height measurements
height_count_path = os.path.join(td, 'lvis-height-count.tif')
rasterize_options = gdal.RasterizeOptions(
    format = 'GTiff',
    outputType = gdal.GDT_Float32,
    creationOptions = ['COMPRESS=DEFLATE', 'TILED=YES'],
    noData = nodata,
    initValues = 0,
    xRes = raster.xps,
    yRes = np.abs(raster.yps),
    outputBounds = [
        raster.xmin,
        raster.ymin,
        raster.xmax,
        raster.ymax
    ],
    burnValues = [1],
    options = ['-add']
)

# then run the rasterization command
print('computing count of tree height values')
print('estimated time: 30s')
ref = gdal.Rasterize(
    height_count_path,
    vector_path,
    options = rasterize_options
)

# and write to disk
ref.FlushCache()
print('done!')

In [ ]:
# finally, divide the summed height by the number of measurements to get the average
tree_height_path = os.path.join(wd, 'tree-height-lvis.tif')
creation_options = "&gdal:co:COMPRESS=DEFLATE&gdal:co:TILED=YES"
output_file = '{file}?{options}'.format(file = tree_height_path, options = creation_options)

# create the band math expression
expression = "(im1b1 > 0 ? im1b1 / im2b1 : {nodata})".format(nodata = nodata)

# set up the orfeo toolbox command
band_math = otb.Registry.CreateApplication("BandMath")
band_math.SetParameterStringList("il", [height_sum_path, height_count_path])
band_math.SetParameterString("out", output_file)
band_math.SetParameterString("exp", expression)

# run the command
band_math.ExecuteAndWriteOutput()

In [ ]:
# then manually set the no-data value
ref = gdal.Open(tree_height_path, gdal.GA_Update)
band = ref.GetRasterBand(1)
band.SetNoDataValue(nodata)
band.FlushCache()
ref.FlushCache()
band = None
ref = None

## Stacking and masking covariate data
Since there are different no-data values, and different pixels with no-data, we'll consistently stack and mask all the data into a single big file. Hooray!

### First, generate a comprehensive mask for all covariate data

In [ ]:
# generate the tree cover mask
tree_cover_path = os.path.join(wd, 'tree-cover.tif')
tree_cover_mask = os.path.join(td, 'tree-cover-mask.tif?')
output_raster = '{file}?{options}'.format(file = tree_cover_mask, options = creation_options)

# set the otb command
masking = otb.Registry.CreateApplication("ManageNoData")
masking.SetParameterString("in", tree_cover_path)
masking.SetParameterString("out", tree_cover_mask)
masking.SetParameterOutputImagePixelType("out", 1)
masking.SetParameterString("mode", "buildmask")

# run it
masking.ExecuteAndWriteOutput()

In [ ]:
# generate the fractional cover mask
fractional_cover_path = os.path.join(wd, 'fcover.tif')
fractional_cover_mask = os.path.join(td, 'fcover-mask.tif?')
output_raster = '{file}?{options}'.format(file = fractional_cover_mask, options = creation_options)

# set the otb command
masking = otb.Registry.CreateApplication("ManageNoData")
masking.SetParameterString("in", fractional_cover_path)
masking.SetParameterString("out", fractional_cover_mask)
masking.SetParameterOutputImagePixelType("out", 1)
masking.SetParameterString("mode", "buildmask")

# run it
masking.ExecuteAndWriteOutput()

In [ ]:
# generate the radar mask
radar_path = os.path.join(wd, 'radar.tif')
radar_mask = os.path.join(td, 'radar-mask.tif?')
output_raster = '{file}?{options}'.format(file = radar_mask, options = creation_options)

# set the otb command
masking = otb.Registry.CreateApplication("ManageNoData")
masking.SetParameterString("in", radar_path)
masking.SetParameterString("out", radar_mask)
masking.SetParameterOutputImagePixelType("out", 1)
masking.SetParameterString("mode", "buildmask")
masking.SetParameterValue("usenan", True)

# run it
masking.ExecuteAndWriteOutput()

In [ ]:
# and a mask for extreme radar values
extreme_mask = os.path.join(td, 'radar-mask-extreme.tif?')
output_raster = '{file}?{options}'.format(file = extreme_mask, options = creation_options)

expression = "(im1b2 < -100 ? 0 : 1) * (im1b1 > 100 ? 0 : 1)"

band_math = otb.Registry.CreateApplication("BandMath")
band_math.SetParameterStringList("il", [radar_path])
band_math.SetParameterString("out", output_raster)
masking.SetParameterOutputImagePixelType("out", 1)
band_math.SetParameterString("exp", expression)

band_math.ExecuteAndWriteOutput()

In [ ]:
# now we'll create a single, comprehensive mask to apply
final_mask = os.path.join(td, 'joint-mask.tif')
output_raster = '{file}?{options}'.format(file = final_mask, options = creation_options)
mask_files = [tree_cover_mask, radar_mask, extreme_mask]

# auto generate the expression
ims = ["im{}b1".format(i + 1) for i in range(len(mask_files))]
expression = " * ".join(ims)

# create and run the otb app
band_math = otb.Registry.CreateApplication("BandMath")
band_math.SetParameterStringList("il", mask_files)
band_math.SetParameterString("out", output_raster)
masking.SetParameterOutputImagePixelType("out", 1)
band_math.SetParameterString("exp", expression)

band_math.ExecuteAndWriteOutput()

### Next, stack all covariate data into a single file and mask it
This should take ~1 minute per cell

In [ ]:
covariate_paths = [radar_path, tree_cover_path, fractional_cover_path]
stacked_path = os.path.join(td, 'covariate-stack.tif')
output_raster = '{file}?{options}'.format(file = stacked_path, options = creation_options)

# otb
concatenate = otb.Registry.CreateApplication("ConcatenateImages")
concatenate.SetParameterStringList("il", covariate_paths)
concatenate.SetParameterString("out", output_raster)

concatenate.ExecuteAndWriteOutput()

# and manually set the no-data value
ref = gdal.Open(stacked_path, gdal.GA_Update)
band = ref.GetRasterBand(1)
band.SetNoDataValue(nodata)
band.FlushCache()
ref.FlushCache()
band = None
ref = None

In [ ]:
# and manually set the no-data value
ref = gdal.Open(stacked_path, gdal.GA_Update)
band = ref.GetRasterBand(1)
band.SetNoDataValue(nodata)
band.FlushCache()
ref.FlushCache()
band = None
ref = None

In [ ]:
# then apply the full mask to the stacked covariates
masked_path = os.path.join(wd, 'covariate-stack.tif')
output_raster = '{file}?{options}'.format(file = masked_path, options = creation_options)

masking = otb.Registry.CreateApplication("ManageNoData")
masking.SetParameterString("in", stacked_path)
masking.SetParameterString("out", output_raster)
masking.SetParameterString("mode", "apply")
masking.SetParameterString("mode.apply.mask", final_mask)

# run it
masking.ExecuteAndWriteOutput()

## Apply the covariate mask to the LVIS average tree height data
This saves us time in reading/masking later in the `modeling` notebook

In [ ]:
tree_height_path = os.path.join(wd, 'tree-height-lvis.tif')

# set the output path
tree_height_masked = tree_height_path[:-4] + '-masked.tif'
output_raster = '{file}?{options}'.format(file = tree_height_masked, options = creation_options)

# set up otb
masking = otb.Registry.CreateApplication("ManageNoData")
masking.SetParameterString("in", tree_height_path)
masking.SetParameterString("out", output_raster)
masking.SetParameterString("mode", "apply")
masking.SetParameterString("mode.apply.mask", final_mask)

# run it
masking.ExecuteAndWriteOutput()

## Split the stacked covariate data into tiles
Since the covariate dataset is very large, applying the model in-memory won't work unless you're sporting some beefy RAM. 

To get around this, we'll chunk the data up into smaller tiles and apply the model to those instead.

In [ ]:
# create a tile directory
tile_directory = os.path.join(wd, 'tiles')
if not os.path.exists(tile_directory):
    os.mkdir(tile_directory)

In [ ]:
# calculate how many tiles we have to create
width = raster.xmax - raster.xmin
height = raster.ymax - raster.ymin
nx_tiles = np.floor(width / tile_size)
ny_tiles = np.floor(height / tile_size)

# set tile lower left coordinates in geographic space
xmins = (np.arange(nx_tiles) * tile_size) + raster.xmin
ymins = (np.arange(ny_tiles) * tile_size) + raster.ymin

In [ ]:
# loop through each of these lower left corners, set the tile boundaries, and use gdal warp to clip each one out
for x, xmin in enumerate(xmins):
    for y, ymin in enumerate(ymins):
        
        # report progress
        if (x % 50 == 0) and (y % 50 == 0):
            print("Running tile x: {x:03d} / {nx:03d} and y: {y:03d} / {ny:03d}".format(
                x = int(x), 
                y = int(y),
                nx = int(len(xmins)),
                ny = int(len(ymins))
            ))
            
        # set the tile name
        warp_output = os.path.join(tile_directory, "covariates-{x:03d}-{y:03d}.tif".format(x = int(x), y = int(y)))
        
        # set the max tile bounds
        xmax = xmin + tile_size
        ymax = ymin + tile_size
        
        # set the warp options
        warp_options = gdal.WarpOptions(
            format = 'GTiff',
            creationOptions = ['COMPRESS=DEFLATE', 'TILED=YES'],
            outputBounds = [
                xmin,
                ymin,
                xmax,
                ymax
            ],
            resampleAlg = gdal.GRA_NearestNeighbour
        )
        
        # run the operation
        gdal.Warp(
            warp_output,
            masked_path,
            options = warp_options
        )

In [ ]:
# then make a pass on each file and delete it if there are no good data values inside
for x, xmin in enumerate(xmins):
    for y, ymin in enumerate(ymins):
        
        # get the file name
        tile_path = os.path.join(tile_directory, "covariates-{x:03d}-{y:03d}.tif".format(x = int(x), y = int(y)))
        
        # read the data into memory
        tile = ccb.read.raster(tile_path)
        tile.read_band(1)
        
        # see if every value is no-data, and delete if so
        nd = tile.data == tile.nodata
        if nd.sum() == (tile.nx * tile.ny):
            os.remove(tile_path)